In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('./Downloads/twelve_five.csv')

print(df.shape)

df.head()

(11198, 14)


,From,Message,Subject,Tags,UID,text,tag_list,first_tag,sender_name,sender_email,domain_name,isNoReply,link_count,clean_text
0,<grangepayments@westernunionspeedpay.com>,"Dear AVRAHAM JACOBSOHN, This is to confirm th...",Grange Payment Confirmation,Finance,31780,<grangepayments@westernunionspeedpay.com> Gran...,['Finance'],Finance,NaN,grangepayments@westernunionspeedpay.com,westernunionspeedpay,False,0,Dear AVRAHAM JACOBSOHN This is to confirm that...
1,Chase <no.reply.alerts@chase.com>,This is an Alert to help manage your account ...,Your Debit Card Transaction,Finance,31779,Chase <no.reply.alerts@chase.com> Your Debit C...,['Finance'],Finance,Chase,no.reply.alerts@chase.com,chase,True,0,This is an Alert to help manage your account e...
2,Amazon Web Services <no-reply-aws@amazon.com>,Please let us know if we helped resolve your i...,Resolved 6559329691: Limit Increase: SageMaker,Productivity,31738,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True,3,Please let us know if we helped resolve your i...
3,Lambda Labs <noreply@github.com>,Youve been added to the Labs 18 - Tagger team ...,Bernie Durfee added you to the Lambda Labs tea...,Productivity,31693,Lambda Labs <noreply@github.com> Bernie Durfee...,['Productivity'],Productivity,Lambda Labs,noreply@github.com,github,True,2,Youve been added to the Labs Tagger team for t...
4,Amazon Web Services <no-reply-aws@amazon.com>,"Hello, We haven't heard back from you regard...",Attention required on case 6559329691: Limit I...,Productivity,31684,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True,2,Hello We haven't heard back from you regarding...


In [10]:
df.dropna(subset=['clean_text'], inplace=True)

In [11]:
df['is_personal'] = df['Tags'].str.contains('Personal')
df.is_personal.value_counts()

False    9779
True     1335
Name: is_personal, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train, test = train_test_split(df, stratify=df['is_personal'])

train.shape, test.shape

((8335, 15), (2779, 15))

In [16]:
X_train = train['clean_text']
y_train = train['is_personal']

X_test = test['clean_text']
y_test = test['is_personal']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8335,), (8335,), (2779,), (2779,))

In [17]:
vect = TfidfVectorizer(stop_words='english')

X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

X_train.shape, X_test.shape

((8335, 418436), (2779, 418436))

In [18]:
model = RandomForestClassifier(n_estimators=500, max_depth=10)

model.fit(X_train, y_train)

print('Model accuracy score: ', model.score(X_test, y_test))

preds = model.predict(X_test)

Model accuracy score:  0.8823317740194314


In [20]:
test['personal_prediction'] = preds

test.head()

/home/jay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,From,Message,Subject,Tags,UID,text,tag_list,first_tag,sender_name,sender_email,domain_name,isNoReply,link_count,clean_text,is_personal,personal_prediction
2063,BC Bursar <BCBursar@brooklyn.cuny.edu>,Dear AVRAHAM JACOBSOHN: \n\nEnroll now for CUN...,Enroll now in the CUNY Scholar Support Card,Other,4743,BC Bursar <BCBursar@brooklyn.cuny.edu> Enroll ...,['Other'],Other,BC Bursar,BCBursar@brooklyn.cuny.edu,brooklyn,False,1,Dear AVRAHAM JACOBSOHN Enroll now for CUNY's S...,False,False
2459,"""PocketFives"" <dancypra@gmail.com>","\nDear thekill21,\n\nThank you for registering...",Action Required to Activate Membership for Poc...,Entertainment,9876,"""PocketFives"" <dancypra@gmail.com> Action Requ...",['Entertainment'],Entertainment,PocketFives,dancypra@gmail.com,gmail,False,4,Dear thekill Thank you for registering at the ...,False,False
8303,YouTube <noreply@youtube.com>,"\r\nYour Personal YouTube Digest - Dec 31, 201...","Your Personal YouTube Digest - Dec 31, 2011",Entertainment,1342,YouTube <noreply@youtube.com> Your Personal Yo...,['Entertainment'],Entertainment,YouTube,noreply@youtube.com,youtube,True,10,Your Personal YouTube Digest Dec Here are the ...,False,False
2666,Amazon Payments <no-reply@amazon.com>,"Greetings from Amazon Payments,\n\n\nYou recei...",You received $1.00 from Kalman Jacobsohn sent ...,Finance,22153,Amazon Payments <no-reply@amazon.com> You rece...,['Finance'],Finance,Amazon Payments,no-reply@amazon.com,amazon,True,1,Greetings from Amazon Payments You received a ...,False,False
3735,Amazon Payments <no-reply@amazon.com>,"Greetings from Amazon Payments,\n\n\nYour paym...",Your payment to Kalman Jacobsohn has failed.,Finance,23460,Amazon Payments <no-reply@amazon.com> Your pay...,['Finance'],Finance,Amazon Payments,no-reply@amazon.com,amazon,True,1,Greetings from Amazon Payments Your payment of...,False,False


In [22]:
right = test['is_personal'] == test['personal_prediction']
#wrong = test['is_personal'] != test['personal_prediction']

bad_preds = test[~right]
good_preds = test[right]

In [23]:
bad_preds.shape

(327, 16)

In [25]:
bad_preds[['From', 'Subject', 'clean_text', 'is_personal', 'personal_prediction']].head(75)

,From,Subject,clean_text,is_personal,personal_prediction
9204,"""Chave Jacobsohn"" <chave@designsbyfmc.com>",RE: Today,Thanks From Avraham Jacobsohn mailto avraham j...,True,False
9530,Yossi Pasternak <yossipasternak@gmail.com>,It's a boy!!,Dear All Mazel Tov With the help of Hashem Reb...,True,False
356,Dede <dede@buildingblockstherapy.org>,Nosson school yr,Hi Can you work with Nosson at his home after ...,True,False
10109,Nikki Avendano <n.avendano1025@gmail.com>,Fwd: TDP-43 papers - Next Wednesday,Begin forwarded message From Avendano Nicole M...,True,False
1838,"""Chave Jacobsohn"" <chave@designsbyfmc.com>",RE: Hi,Is it a good movie But you don't have cheese b...,True,False
...,...,...,...,...,...
1235,william_chao <e8132638@netvigator.com>,=?Big5?B?pl7C0KFH?=,Dear mr Jacobson thanks for the e mail you can...,True,False
2153,"""Chave Jacobsohn"" <chave@designsbyfmc.com>",RE: Hi,When do you finish class I forgot you had clas...,True,False
10161,Laurie Cole <laurie@kworthins.com>,RE: from Laurie at Killingsworth,QXdlc tZSwgVGhhbmt b UhDQoNCg KDQpMYXVyaWUgQ s...,True,False
1808,kalman <ksemail7@gmail.com>,Re:,did you take care of dropping that class yet i...,True,False


In [26]:
train_csv = df[['clean_text', 'is_personal']]

In [27]:
df.to_csv('twelve_twelve_update.csv', index=False)

In [28]:
train.columns

Index(['From', 'Message', 'Subject', 'Tags', 'UID', 'text', 'tag_list',
       'first_tag', 'sender_name', 'sender_email', 'domain_name', 'isNoReply',
       'link_count', 'clean_text', 'is_personal'],
      dtype='object')

In [30]:
df.domain_name.unique()

array(['westernunionspeedpay', 'chase', 'amazon', 'github', 'accounts',
       'statefarm', 'login', 'samharris', 'acctmanagement', 'emailonline',
       'ku', 'mail', 'alertsp', 'freecodecamp', 'ebird', 'npmjs', 'gmail',
       'alephbeis', 'buildingblockstherapy', 'pointandpay',
       'starsaccount', 'united', 'zoom', 'grangeinsurance',
       'escaperoommaster', 'designsbyfmc', 'bacb', 'yahoo', 'aol',
       'kidslinkohio', 'googlemail', 'cuyahogacounty', 'pokerstars',
       'blackchippoker', 'pocketfives', 'reddit', 'shaw', 'correo',
       'houstonisd', 'preferredservicesny', 'wowway', 'carbonpoker',
       'zo7d5i', 'fulltiltpoker', 'applicationassist', 'lockpoker',
       'chainloopstrategy', 'apps', 'modernmediahost', 'completelogics',
       'powerfulconnection', 'specificinterval', 'waxdouble',
       'pokerfraudalert', 'bluffmedia', 'email', 'facebookmail',
       'ipetitions', 'chiangsman', 'betcris', 'rocketmail', 'dos',
       'smarter-apps', 'google', 'tmo', 'fivesmail

In [31]:
df.domain_name.nunique()

411

In [35]:
top_ten = df.domain_name.value_counts()[:10]

In [36]:
top_ten

amazon          5163
jdeal            752
designsbyfmc     655
pokerstars       548
twoplustwo       449
brooklyn         267
uw               260
gmail            259
alertsp          229
e                120
Name: domain_name, dtype: int64

In [39]:
df[df['domain_name'] == 'e']

,From,Message,Subject,Tags,UID,text,tag_list,first_tag,sender_name,sender_email,domain_name,isNoReply,link_count,clean_text,is_personal
1610,"""Udemy Instructor: Eduonix Learning Solutions""...","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",Grand Sale! Grab Best Web Dev Courses At Just ...,Entertainment,68494,"""Udemy Instructor: Eduonix Learning Solutions""...",['Entertainment'],Entertainment,Udemy Instructor: Eduonix Learning Solutions,no-reply@e.udemymail.com,e,True,40,DOCTYPE HTML PUBLIC W C DTD HTML EN http www w...,False
1616,"""Crystal Springs Resort"" <csr@e.crystalgolfres...",Golf Vacation at the Northeast's Most Acclaime...,=?UTF-8?B?8J+PjO+4jyBWYWNhdGlvbiBhdCB0aGUgTm9y...,Finance,68288,"""Crystal Springs Resort"" <csr@e.crystalgolfres...",['Finance'],Finance,Crystal Springs Resort,csr@e.crystalgolfresort.com,e,False,32,Golf Vacation at the Northeast's Most Acclaime...,False
1619,=?UTF-8?B?TG93ZeKAmXMgSG9tZSBJbXByb3ZlbWVudA==...,View Web Version: \nhttps://view.e.lowes.com/?...,Start Your Next Outdoor Project with Up to 25%...,Finance,68047,=?UTF-8?B?TG93ZeKAmXMgSG9tZSBJbXByb3ZlbWVudA==...,['Finance'],Finance,=?UTF-8?B?TG93ZeKAmXMgSG9tZSBJbXByb3ZlbWVudA==...,lowes@e.lowes.com,e,False,4,View Web Version https view e lowes com qs d a...,False
1620,"""Crystal Springs Resort"" <csr@e.crystalgolfres...",Why is New Jersey Called the Garden State? \n\...,=?UTF-8?B?V2h5IGlzIE5ldyBKZXJzZXkgQ2FsbGVkIHRo...,Finance,68017,"""Crystal Springs Resort"" <csr@e.crystalgolfres...",['Finance'],Finance,Crystal Springs Resort,csr@e.crystalgolfresort.com,e,False,26,Why is New Jersey Called the Garden State http...,False
1699,"""Crystal Springs Resort"" <csr@e.crystalgolfres...",Free 2-Day Shipping on Resort Gift Cards \n\nh...,Free 2-Day Shipping on Resort Gift Cards,Other,64872,"""Crystal Springs Resort"" <csr@e.crystalgolfres...",['Other'],Other,Crystal Springs Resort,csr@e.crystalgolfresort.com,e,False,23,Free Day Shipping on Resort Gift Cards http cl...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10036,Chase <Chase@e.chase.com>,<!--Assign codes for use with unsubscribe link...,"John, check out your new Chase Offers",Finance,1280,"Chase <Chase@e.chase.com> John, check out you...",['Finance'],Finance,Chase,Chase@e.chase.com,e,False,17,Assign codes for use with unsubscribe links in...,False
10101,Chase <Chase@e.chase.com>,\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html PUBLIC ...,"JOHN, activate your Chase Offers before they'r...",Finance,1346,"Chase <Chase@e.chase.com> JOHN, activate your...",['Finance'],Finance,Chase,Chase@e.chase.com,e,False,51,DOCTYPE html PUBLIC W C DTD XHTML E Transition...,False
10144,"""NORDSTROM RACK"" <nordstromrack@e.nordstromrac...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n Shop luxury t...,The MICHELE Watches Event starts now,Shopping,1389,"""NORDSTROM RACK"" <nordstromrack@e.nordstromrac...",['Shopping'],Shopping,NORDSTROM RACK,nordstromrack@e.nordstromrack.com,e,False,39,Shop luxury timepieces for her mdash; hours on...,False
10145,"""NORDSTROM RACK"" <nordstromrack@e.nordstromrac...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n Shop activewe...,The Outdoor Voices Event starts now,Shopping,1390,"""NORDSTROM RACK"" <nordstromrack@e.nordstromrac...",['Shopping'],Shopping,NORDSTROM RACK,nordstromrack@e.nordstromrack.com,e,False,35,Shop activewear swim mdash; days only Shop act...,False


In [43]:
df['From'][1610].split('@')[-1]

'e.udemymail.com>'

In [ ]:
df['domain_name'] = df['From'].

In [44]:
def get_domain_name(val):
    domain = val.split('@')[-1]
    domain_list = domain.split('.')
    return domain_list[-2]

In [45]:
df['domain_name'] = df['From'].apply(get_domain_name)

In [47]:
df['domain_name'].value_counts()

amazon          5170
jdeal            752
designsbyfmc     655
pokerstars       548
twoplustwo       449
                ... 
wcs                1
hawthornes         1
sforumsmail        1
ALSAFL             1
udemymail          1
Name: domain_name, Length: 414, dtype: int64

In [48]:
df['domain_name'].nunique()

414

In [49]:
top_twenty = df['domain_name'].value_counts()[:20]

In [50]:
top_twenty

amazon                   5170
jdeal                     752
designsbyfmc              655
pokerstars                548
twoplustwo                449
chase                     269
cuny                      269
uw                        260
gmail                     259
linkedin                  104
nordstrom                  99
carbonpoker                99
starsaccount               89
buildingblockstherapy      72
slack                      67
google                     60
freecodecamp               51
fulltiltpoker              48
youtube                    46
capitalone                 45
Name: domain_name, dtype: int64

In [52]:
import numpy as np
import gensim

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [53]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [54]:
df['tokens'] = [tokenize(text) for text in df['clean_text']]

In [55]:
id2word = corpora.Dictionary(df['tokens'])

In [63]:
len(id2word.keys())

400963

In [64]:
id2word.filter_extremes(no_below=10, no_above=0.75)
len(id2word.keys())

7893

In [65]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()